# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [5]:
# Your code here
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite');

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [11]:
# Your code here
pd.read_sql('''
SELECT lastName, firstName, city
FROM employees
JOIN offices
USING(officeCode)
WHERE city = 'Boston'
''', conn)

,lastName,firstName,city
0,Firrelli,Julie,Boston
1,Patterson,Steve,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [24]:
# Your code here
pd.read_sql('''
SELECT officeCode, city,
COUNT(employeeNumber) AS number_of_employees
FROM offices
LEFT JOIN employees
USING(officeCode)
GROUP BY officeCode
HAVING number_of_employees = 0
''', conn)

,officeCode,city,number_of_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [29]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
pd.read_sql('''
SELECT o.officeCode, o.city, 
       COUNT(c.customerNumber) AS number_of_customers
FROM offices o
LEFT JOIN employees e USING(officeCode)
LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY o.officeCode, o.city
''', conn)

,officeCode,city,number_of_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17
7,27,Boston,0


In [33]:
"""
Question 1
What are the top 5 cities with the most customers
"""
pd.read_sql('''
SELECT city,
COUNT(customerNumber) AS number_of_customers
FROM customers
GROUP BY city
ORDER BY number_of_customers DESC
LIMIT 5

''', conn)

,city,number_of_customers
0,NYC,5
1,Madrid,5
2,Singapore,3
3,Paris,3
4,Brickhaven,3


In [37]:
"""
Question 2
Which employees have never made a sale
"""

# Your code here
pd.read_sql('''
SELECT e.firstName, e.lastName, e.employeeNumber
FROM employees e
LEFT JOIN customers c
ON e.employeeNumber = c.salesRepEmployeeNumber
WHERE c.customerNumber = NULL
''', conn)

,firstName,lastName,employeeNumber


In [39]:
"""
Question 3
Which office has the highest number of employees
"""

# Your code here
pd.read_sql('''
SELECT officeCode, city,
COUNT(employeeNumber) AS numer_of_employees
FROM employees
LEFT JOIN offices
USING(officeCode)
GROUP BY officeCode

''',conn)

,officeCode,city,numer_of_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [42]:
# Your code here
pd.read_sql('''
SELECT e.firstName, e.lastName, p.productName
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderDetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
ORDER BY e.lastName, e.firstName, p.productName
''', conn)

,firstName,lastName,productName
0,Loui,Bondur,18th century schooner
1,Loui,Bondur,1900s Vintage Bi-Plane
2,Loui,Bondur,1900s Vintage Tri-Plane
3,Loui,Bondur,1903 Ford Model A
4,Loui,Bondur,1903 Ford Model A
...,...,...,...
2991,George,Vanauf,The Titanic
2992,George,Vanauf,The Titanic
2993,George,Vanauf,The USS Constitution Ship
2994,George,Vanauf,The USS Constitution Ship


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [47]:
# Your code here
pd.read_sql('''
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(p.productCode) AS number_of_products
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderDetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
GROUP BY e.employeeNumber
''', conn)

,employeeNumber,firstName,lastName,number_of_products
0,1165,Leslie,Jennings,331
1,1166,Leslie,Thompson,114
2,1188,Julie,Firrelli,124
3,1216,Steve,Patterson,152
4,1286,Foon Yue,Tseng,142
5,1323,George,Vanauf,211
6,1337,Loui,Bondur,177
7,1370,Gerard,Hernandez,396
8,1401,Pamela,Castillo,272
9,1501,Larry,Bott,236


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [48]:
# Your code here
pd.read_sql('''
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(p.productCode) AS number_of_products
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderDetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
GROUP BY e.employeeNumber
HAVING number_of_products > 200
''', conn)

,employeeNumber,firstName,lastName,number_of_products
0,1165,Leslie,Jennings,331
1,1323,George,Vanauf,211
2,1370,Gerard,Hernandez,396
3,1401,Pamela,Castillo,272
4,1501,Larry,Bott,236
5,1504,Barry,Jones,220


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!